In [1]:
import time

import gym
import accelerator_environments
import cheetah
import matplotlib.pyplot as plt
import numpy as np
import ocelot as oc

import accelerator_environments.envs.ares.ARESlatticeStage3v1_9 as aresstuff
from accelerator_environments import utils

initializing ocelot...


In [2]:
binning = 4
screen_resolution = (int(2448 / binning), int(2040 / binning))
meter_per_pixel = (3.5488e-6 * binning, 2.5003e-6 * binning)
screen_bin_edges = (np.linspace(-screen_resolution[0]*meter_per_pixel[0]/2,
                                             screen_resolution[0]*meter_per_pixel[0]/2,
                                             screen_resolution[0]+1),
                                 np.linspace(-screen_resolution[1]*meter_per_pixel[1]/2,
                                             screen_resolution[1]*meter_per_pixel[1]/2,
                                             screen_resolution[1]+1))


cell = utils.subcell_of(aresstuff.cell, "AREASOLA1", "AREABSCR1")
aresstuff.areamqzm1.k1 = 5.5
aresstuff.areamqzm2.k1 = -5.5
aresstuff.areamqzm3.k1 = 0.0
# cell = [oc.Drift(l=3.0), oc.Quadrupole(l=1, k1=0), oc.Drift(l=0), oc.Drift(l=3)]
# cell = [oc.Drift(l=3.0)]

lattice = oc.MagneticLattice(cell)

In [3]:
particles = oc.generate_parray(nparticles=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

navigator = oc.Navigator(lattice)
_, particles = oc.track(lattice, particles, navigator)
image = np.histogram2d(particles.x(), particles.y(),
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

z = 2.0460399999999996 / 2.0460399999999996 : applied: 0.33204102516174316 seconds


In [4]:
particles = oc.generate_parray(nparticles=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

navigator = oc.Navigator(lattice)
navigator.unit_step = lattice.totalLen
_, particles = oc.track(lattice, particles, navigator)
image = np.histogram2d(particles.x(), particles.y(),
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

z = 2.0460399999999996 / 2.0460399999999996 : applied: 0.4584038257598877 seconds


In [7]:
segment = cheetah.Segment.from_ocelot(cell)
segment.AREABSCR1.is_active = True  # Turn screen on and off

particles = cheetah.Beam.make_random(n=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

_ = segment(particles)
image = segment.AREABSCR1.reading

t2 = time.time()

print(f"{t2 - t1} seconds")

Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.Quadrupole'> on device cpu
Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.Quadrupole'> on device cpu
Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.VerticalCorrector'> on device cpu
Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.Quadrupole'> on device cpu
Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.HorizontalCorrector'> on device cpu
Element of type <class 'cheetah.accelerator.Drift'> on device cpu
Element of type <class 'cheetah.accelerator.Screen'> on device cpu
Element of type <class 'cheetah.accelerator.Segment'> on device cpu
Element of type <class 'cheetah.